# This Notebook will be used for Coursera Capstone Project of IBM Data Science

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Web Scraping Data for Neighbourhoods in Torronto

In [3]:
from bs4 import BeautifulSoup
import requests

In [4]:
r= requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r.text)

In [30]:
table = soup.find_all("table")[0]
data = []
temp = []

for td in table.find_all('td'):
    temp.append(td.get_text().rstrip())
    if(len(temp) == 3):
        if(temp[1] != 'Not assigned'):
            #checking if previous added entry has same PostalCode
            if(len(data) > 0 and data[len(data)-1][0] == temp[0]):
                data[len(data)-1][2] = temp[2] + ", " + data[len(data)-1][2]
            else:
                data.append(temp)
        last=temp
        temp = []
df = pd.DataFrame(data, columns = ['PostalCode', 'Borough', 'Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Not assigned


In [25]:
df.shape

(103, 3)

## Adding Latitutde and Longitude 

In [6]:
df2 = pd.read_csv("https://cocl.us/Geospatial_data")

In [28]:
df2.rename(columns= {"Postal Code": "PostalCode"}, inplace=True)
df2.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [33]:
df3 = pd.merge(df, df2, how='inner', on='PostalCode')
df3.head(11)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Segmenting and Clustering Neighbourhoods

In [61]:
import folium
from pandas.io.json import json_normalize

Only work with data that has Toronto in Borough

In [40]:
li = ['Downtown Toronto', 'East Toronto' , 'West Toronto' , 'Central Toronto']
df4 = df3[df3.Borough.isin(li)]
df4.reset_index(inplace=True, drop=True)
df4.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Creating Map of Toronto

In [52]:
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

for lat, lng, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [53]:
CLIENT_ID = '3IUT4PW5215DP5MLXBOK1BO2DR2PSAZBVSLHQBMIZDPSQCSV' # your Foursquare ID
CLIENT_SECRET = 'DMQF43K2IS42VOO3CXP5YLYZMU1FHSYPF231FV2VWPUTPYBF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [54]:
df4.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [57]:
neighborhood_latitude = df4.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df4.loc[0, 'Longitude'] # neighborhood longitude value
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [58]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d94396e66fc65002cb309b7'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 51,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

In [63]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [67]:
toronto_venues = getNearbyVenues(names=df4['Neighborhood'],latitudes=df4['Latitude'],longitudes=df4['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, King, Adelaide
Dufferin, Dovercourt Village
Union Station, Toronto Islands, Harbourfront East
Trinity, Little Portugal
Riverdale, The Danforth West
Toronto Dominion Centre, Design Exchange
Parkdale Village, Exhibition Place, Brockton
India Bazaar, The Beaches West
Victoria Hotel, Commerce Court
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill West, Forest Hill North
The Junction South, High Park
North Toronto West
Yorkville, North Midtown, The Annex
Roncesvalles, Parkdale
Davisville
University of Toronto, Harbord
Swansea, Runnymede
Summerhill East, Moore Park
Kensington Market, Grange Park, Chinatown
Summerhill West, South Hill, Rathnelly, Forest Hill SE, Deer Park
South Niagara, Railway Lands, King and Spadina, Harbourfront West, Island airport, Bathurst Quay, CN Tower
Rosedale
Stn A PO Boxes 25 The Esplanade
St. James Town, Cabbagetown
Und

In [68]:
print(toronto_venues.shape)
toronto_venues.head()

(1709, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [69]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
Central Bay Street,84,84,84,84,84,84
Christie,15,15,15,15,15,15
Church and Wellesley,92,92,92,92,92,92
Davisville,36,36,36,36,36,36
Davisville North,8,8,8,8,8,8
"Dufferin, Dovercourt Village",16,16,16,16,16,16
"Forest Hill West, Forest Hill North",4,4,4,4,4,4


### Analyse Each Neighborhood

In [70]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
toronto_onehot.shape

(1709, 236)

In [73]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.00000
1,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000
2,Central Bay Street,0.011905,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,...,0.000000,0.00000,0.000000,0.000000,0.00,0.011905,0.000000,0.000000,0.011905,0.00000
3,Christie,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000
4,Church and Wellesley,0.010870,0.01087,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,...,0.010870,0.01087,0.000000,0.000000,0.00,0.000000,0.010870,0.010870,0.000000,0.01087
5,Davisville,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.027778,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000
6,Davisville North,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000
7,"Dufferin, Dovercourt Village",0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000
8,"Forest Hill West, Forest Hill North",0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000
9,"Garden District, Ryerson",0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,...,0.020000,0.00000,0.010000,0.000000,0.00,0.010000,0.010000,0.010000,0.010000,0.00000


#### Let's print each neighborhood along with the top 5 most common venues

In [74]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2              Bakery  0.04
3  Italian Restaurant  0.04
4                Café  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2          Restaurant  0.06
3         Pizza Place  0.06
4          Skate Park  0.06


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.14
1         Italian Restaurant  0.05
2                       Café  0.05
3  Middle Eastern Restaurant  0.04
4               Burger Joint  0.04


----Christie----
               venue  freq
0      Grocery Store  0.20
1               Café  0.20
2               Park  0.13
3  Convenience Store  0.07
4          Nightclub  0.07


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.09
1              Gay Bar  0.05
2  Japanese Restaurant  0.05
3     Sushi Resta

In [75]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [77]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Steakhouse,Cheese Shop,Seafood Restaurant,Café,Italian Restaurant,Farmers Market
1,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Recording Studio,Restaurant
2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant,Burger Joint,Ice Cream Shop,Salad Place,Bubble Tea Shop,Bar
3,Christie,Grocery Store,Café,Park,Baby Store,Nightclub,Italian Restaurant,Coffee Shop,Convenience Store,Diner,Restaurant
4,Church and Wellesley,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Men's Store,Gym,Fast Food Restaurant,Smoke Shop,Mediterranean Restaurant


### Cluster Neighbourhoods

In [84]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 4, 0], dtype=int32)

In [87]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [89]:
toronto_merged = df4

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Café,Restaurant,Mexican Restaurant,Breakfast Spot,Theater,Gym / Fitness Center
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Bookstore,Pizza Place,Fast Food Restaurant,Italian Restaurant,Plaza
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Gastropub,Clothing Store,Beer Bar,Breakfast Spot,Cocktail Bar
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Coffee Shop,Trail,Pub,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Steakhouse,Cheese Shop,Seafood Restaurant,Café,Italian Restaurant,Farmers Market


In [91]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [93]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining Clusters

Cluster 1

In [94]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Park,Café,Restaurant,Mexican Restaurant,Breakfast Spot,Theater,Gym / Fitness Center
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Bookstore,Pizza Place,Fast Food Restaurant,Italian Restaurant,Plaza
2,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Gastropub,Clothing Store,Beer Bar,Breakfast Spot,Cocktail Bar
3,East Toronto,0,Health Food Store,Coffee Shop,Trail,Pub,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
4,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Steakhouse,Cheese Shop,Seafood Restaurant,Café,Italian Restaurant,Farmers Market
5,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant,Burger Joint,Ice Cream Shop,Salad Place,Bubble Tea Shop,Bar
6,Downtown Toronto,0,Grocery Store,Café,Park,Baby Store,Nightclub,Italian Restaurant,Coffee Shop,Convenience Store,Diner,Restaurant
7,Downtown Toronto,0,Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Restaurant,Hotel,Burger Joint,American Restaurant,Cosmetics Shop
8,West Toronto,0,Pharmacy,Bakery,Supermarket,Pizza Place,Athletics & Sports,Middle Eastern Restaurant,Music Venue,Park,Café,Brewery
9,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Brewery,Fried Chicken Joint,Scenic Lookout,Sports Bar,Bakery,Pizza Place


Cluster 2

In [95]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1,Ice Cream Shop,Garden,Music Venue,Wings Joint,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Cluster 3

In [96]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Central Toronto,2,Playground,Park,Tennis Court,Wings Joint,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
32,Downtown Toronto,2,Park,Playground,Trail,Building,Wings Joint,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant


Cluster 4

In [97]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,3,Bus Line,Park,Swim School,Wings Joint,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Cluster 5

In [98]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Central Toronto,4,Trail,Mexican Restaurant,Jewelry Store,Sushi Restaurant,Wings Joint,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
